# First we define a function that, given a pb filename, loads it as a tensorflow graph

In [1]:
import tensorflow as tf
import numpy as np

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

# An example of how to load pb files as graphs, and print their ops

In [23]:
from tensorflow.contrib.quantization import *
# We use our "load_graph" function
# inception_aesthetic = load_graph('../../../aesthetic_learning/inception_model/output_graph.pb')
# small_graph = load_graph('../examples/models/aesthetics_caffenet/aesthetic_classif_quantized.pb')
# full_graph = load_graph('../examples/models/aesthetics_caffenet/frozen_aesthetics_model.pb')
googlenet_graph = load_graph('../examples/models/bvlc_googlenet/frozen_model_googlenet.pb')
# We can verify that we can access the list of operations in the graph
for op in googlenet_graph.get_operations():
    print(op.name)
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions

prefix/Placeholder
prefix/conv1_7x7_s2/weights
prefix/conv1_7x7_s2/weights/read
prefix/conv1_7x7_s2/Conv2D
prefix/conv1_7x7_s2/biases
prefix/conv1_7x7_s2/biases/read
prefix/conv1_7x7_s2/BiasAdd
prefix/conv1_7x7_s2/conv1_7x7_s2
prefix/pool1_3x3_s2
prefix/pool1_norm1
prefix/conv2_3x3_reduce/weights
prefix/conv2_3x3_reduce/weights/read
prefix/conv2_3x3_reduce/Conv2D
prefix/conv2_3x3_reduce/biases
prefix/conv2_3x3_reduce/biases/read
prefix/conv2_3x3_reduce/BiasAdd
prefix/conv2_3x3_reduce/conv2_3x3_reduce
prefix/conv2_3x3/weights
prefix/conv2_3x3/weights/read
prefix/conv2_3x3/Conv2D
prefix/conv2_3x3/biases
prefix/conv2_3x3/biases/read
prefix/conv2_3x3/BiasAdd
prefix/conv2_3x3/conv2_3x3
prefix/conv2_norm2
prefix/pool2_3x3_s2
prefix/inception_3a_1x1/weights
prefix/inception_3a_1x1/weights/read
prefix/inception_3a_1x1/Conv2D
prefix/inception_3a_1x1/biases
prefix/inception_3a_1x1/biases/read
prefix/inception_3a_1x1/BiasAdd
prefix/inception_3a_1x1/inception_3a_1x1
prefix/inception_3a_3x3_reduce/

# An important step: we need to make sure the image we test on has the right format, matching the graph input layer

In [24]:
from scipy import misc
image_np = misc.imread('grace_hopper.jpg')
image_resized = misc.imresize(image_np,(224,224,3))
expanded = np.expand_dims(image_resized,axis=0)
duplicated = np.repeat(expanded,32, axis=0)

# Now we can fill the input tensor with the image, and print out the output (often a vector of probabilities)

In [25]:
import numpy as np
# We access the input and output nodes 
x_small = googlenet_graph.get_tensor_by_name('prefix/Placeholder:0')
y_small = googlenet_graph.get_tensor_by_name('prefix/prob:0')

# We launch a Session
with tf.Session(graph=googlenet_graph) as sess:
    # Note: we didn't initialize/restore anything, everything is stored in the graph_def
    y_out = sess.run(y_small, feed_dict={
        x_small: duplicated # small graph
    })
    print(y_out) # [[ False ]] Yay, it works!
    print(y_out.shape)

[[  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]
 [  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]
 [  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]
 ..., 
 [  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]
 [  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]
 [  7.44015722e-07   7.79389552e-07   5.32602371e-06 ...,   1.08753184e-07
    2.93676317e-06   8.79124946e-06]]
(32, 1000)
